## Exploring Airplane Safety Data

In [41]:
import pandas as pd

In [146]:
!ls

Analysis Outline.ipynb
Aviation Data Exploration.ipynb
CONTRIBUTING.md
Clean_AviationII.csv
Irwin's Analysis.ipynb
LICENSE.md
Phase 1 Project Final.ipynb
Phase of Flight and Safety Score Analysis-Copy1.ipynb
Phase of Flight and Safety Score Analysis.ipynb
README.md
Years and Purpose .ipynb
data
example_dashboard.png
student.ipynb


## Read CSV

In [147]:
df = pd.read_csv('data/Aviation_data.csv')

/Users/kaceyclougher/anaconda3/envs/learn-env/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3145: DtypeWarning: Columns (6,7,28) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [148]:
#Initial inspection shows a large number of Null values in each column

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90348 entries, 0 to 90347
Data columns (total 31 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Event.Id                88889 non-null  object 
 1   Investigation.Type      90348 non-null  object 
 2   Accident.Number         88889 non-null  object 
 3   Event.Date              88889 non-null  object 
 4   Location                88837 non-null  object 
 5   Country                 88663 non-null  object 
 6   Latitude                34382 non-null  object 
 7   Longitude               34373 non-null  object 
 8   Airport.Code            50249 non-null  object 
 9   Airport.Name            52790 non-null  object 
 10  Injury.Severity         87889 non-null  object 
 11  Aircraft.damage         85695 non-null  object 
 12  Aircraft.Category       32287 non-null  object 
 13  Registration.Number     87572 non-null  object 
 14  Make                    88826 non-null

In [149]:
#Initial numerical data investigation

df.describe()

,Number.of.Engines,Total.Fatal.Injuries,Total.Serious.Injuries,Total.Minor.Injuries,Total.Uninjured
count,82805.000000,77488.000000,76379.000000,76956.000000,82977.000000
mean,1.146585,0.647855,0.279881,0.357061,5.325440
std,0.446510,5.485960,1.544084,2.235625,27.913634
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.000000,0.000000,0.000000,0.000000,0.000000
50%,1.000000,0.000000,0.000000,0.000000,1.000000
75%,1.000000,0.000000,0.000000,0.000000,2.000000
max,8.000000,349.000000,161.000000,380.000000,699.000000


In [150]:
#Nulls values per column 
df.isna().sum()

Event.Id                   1459
Investigation.Type            0
Accident.Number            1459
Event.Date                 1459
Location                   1511
Country                    1685
Latitude                  55966
Longitude                 55975
Airport.Code              40099
Airport.Name              37558
Injury.Severity            2459
Aircraft.damage            4653
Aircraft.Category         58061
Registration.Number        2776
Make                       1522
Model                      1551
Amateur.Built              1561
Number.of.Engines          7543
Engine.Type                8536
FAR.Description           58325
Schedule                  77766
Purpose.of.flight          7651
Air.carrier               73700
Total.Fatal.Injuries      12860
Total.Serious.Injuries    13969
Total.Minor.Injuries      13392
Total.Uninjured            7371
Weather.Condition          5951
Broad.phase.of.flight     28624
Report.Status              7840
Publication.Date          16689
dtype: i

In [151]:
df['Aircraft.Category'].value_counts()

Airplane             27617
Helicopter            3440
Glider                 508
Balloon                231
Gyrocraft              173
Weight-Shift           161
Powered Parachute       91
Ultralight              30
Unknown                 14
WSFT                     9
Powered-Lift             5
Blimp                    4
UNK                      2
ULTR                     1
Rocket                   1
Name: Aircraft.Category, dtype: int64

# Data Cleaning 

In [152]:
#Cols to Drop

df.drop(columns = ['Latitude', 'Longitude', 'Airport.Code', 'Airport.Name', 
                   'Investigation.Type', 'Registration.Number', 'Publication.Date',
                   'FAR.Description', 'Schedule', 'Air.carrier', 'Amateur.Built', 'Report.Status'], inplace = True)

In [153]:
#Change Column names

df.columns = df.columns.str.replace('.', '_')

In [154]:
#Duplicates 

#Drop
df.drop_duplicates(inplace = True)

#Check duplicates have been deleted
df.duplicated().value_counts()

False    88888
dtype: int64

In [155]:
#Event_Id drop

#df.dropna(subset = ['Event_Id'], inplace = True)
df.dropna(subset = ['Accident_Number', 'Event_Date'], inplace = True)


## Goal to make following down the line 0

In [156]:
df.isna().sum() #remove the nulls

Event_Id                      0
Accident_Number               0
Event_Date                    0
Location                     52
Country                     226
Injury_Severity             999
Aircraft_damage            3193
Aircraft_Category         56602
Make                         63
Model                        92
Number_of_Engines          6083
Engine_Type                7075
Purpose_of_flight          6191
Total_Fatal_Injuries      11401
Total_Serious_Injuries    12510
Total_Minor_Injuries      11933
Total_Uninjured            5912
Weather_Condition          4491
Broad_phase_of_flight     27163
dtype: int64

## Cleaning Dates - Done

In [157]:
df['Event_Date'] = pd.to_datetime(df['Event_Date'])

# Extract year, month, and day
df['created_year'] = df['Event_Date'].dt.year
df['created_month'] = df['Event_Date'].dt.month
df['created_day'] = df['Event_Date'].dt.day

## Scrubbing Non-Aircrafts from dataset

In [158]:
#Remove non-aircrafts
df = df[df['Aircraft_Category'] == 'Airplane']

#Check
df['Aircraft_Category'].value_counts()

Airplane    27615
Name: Aircraft_Category, dtype: int64

## Cleaning Weather - Done

In [159]:
df['Weather_Condition'] = df['Weather_Condition'].apply(lambda x: str(x).upper())

In [160]:
df['Weather_Condition'] = df['Weather_Condition'].apply(lambda x : 'UNK' if ((x == 'NAN') | (x == 'Unk')) else x)

In [161]:
df['Weather_Condition'].value_counts()

VMC    22768
UNK     3438
IMC     1409
Name: Weather_Condition, dtype: int64

## Cleaning Injuries - Done

In [162]:
#Cleaning up injuries
df['Total_Fatal_Injuries'].value_counts().unique()

#Replace with mode (0)
df['Total_Fatal_Injuries'].fillna(0, inplace = True)

#Check
df['Total_Fatal_Injuries'].isna().sum()

0

In [163]:
#Cleaning up injuries
df['Total_Serious_Injuries'].value_counts().unique()

#Replace with mode (0)
df['Total_Serious_Injuries'].fillna(0, inplace = True)

#Check
df['Total_Serious_Injuries'].isna().sum()

0

In [164]:
#Cleaning up injuries
df['Total_Minor_Injuries'].value_counts().unique()

#Replace with mode (0)
df['Total_Minor_Injuries'].fillna(0, inplace = True)

#Check
df['Total_Minor_Injuries'].isna().sum()

0

In [165]:
#Cleaning up injuries
df['Total_Uninjured'].value_counts().unique()

#Replace with mode (0)
df['Total_Uninjured'].fillna(0, inplace = True)

#Check
df['Total_Uninjured'].isna().sum()

0

In [166]:
#Check out column
df['Injury_Severity'].value_counts()

#Clean the values from Fatal(#) or just Fatal
# count # of character in column if more than 5 replace with Fatal
df.loc[df['Injury_Severity'].str[:5] == 'Fatal', 'Injury_Severity'] = 'Fatal'

#clean na's
df['Injury_Severity'].fillna('Unavailable', inplace = True)

#check 
df['Injury_Severity'].isna().sum()

0

## Cleaning up Damage - Done

In [167]:
#Rename columns
df.rename(columns = {'Aircraft_damage':'Aircraft_Damage'}, inplace = True)

In [168]:
#change nulls to Unknown Category
df['Aircraft_Damage'].fillna("Unknown", inplace = True)

#check 
df['Aircraft_Damage'].isna().sum()

0

## Cleaning Make

In [169]:
df['Make'].value_counts()

CESSNA                    4867
Cessna                    3608
PIPER                     2805
Piper                     1910
BOEING                    1035
                          ... 
KERR JOHN A                  1
Carlisle                     1
PILATUS BRITTEN-NORMAN       1
Tail Feather LLC             1
HUNTER GEORGE                1
Name: Make, Length: 3874, dtype: int64

In [170]:
#capitalize and merge make models
df['Make'] = df['Make'].apply(lambda x: str(x).capitalize())

In [171]:
#take out all airplanes with make less than 100
df = df[df.groupby('Make')['Make'].transform('count') > 100]

In [172]:
df['Make'].value_counts()

Cessna                8475
Piper                 4715
Beech                 1692
Boeing                1322
Mooney                 419
Bellanca               282
Grumman                251
Airbus                 245
Maule                  232
Aeronca                229
Air tractor            224
Cirrus design corp     220
Air tractor inc        219
Champion               170
Luscombe               164
Embraer                155
Stinson                146
Cirrus                 137
Vans                   125
North american         118
Mcdonnell douglas      115
Taylorcraft            110
Aero commander         107
Name: Make, dtype: int64

In [173]:
#Combine air tractor
df.loc[df['Make'].str[:11] == 'Air tractor', 'Make'] = 'Air tractor'

#Combine Mcdonnell douglas #question for praveen
df.loc[df['Make'].str[:] == 'Mcdonnell douglas', 'Make'] = 'Douglas'

#Check
df['Make'].value_counts()

Cessna                8475
Piper                 4715
Beech                 1692
Boeing                1322
Air tractor            443
Mooney                 419
Bellanca               282
Grumman                251
Airbus                 245
Maule                  232
Aeronca                229
Cirrus design corp     220
Champion               170
Luscombe               164
Embraer                155
Stinson                146
Cirrus                 137
Vans                   125
North american         118
Douglas                115
Taylorcraft            110
Aero commander         107
Name: Make, dtype: int64

In [174]:
#check for nulls
df['Make'].isna().sum()

0

**Model**

In [175]:
#dropping null models
df.dropna(subset = ['Model'], inplace = True)

#check
df['Model'].isna().sum()

0

## Adding Total Injuries

In [176]:
#make total injuries column
df['Total_Injuries'] = (df['Total_Fatal_Injuries'] + 
                                  df['Total_Minor_Injuries'] +
                                 df['Total_Serious_Injuries'])

#check
df.head()

,Event_Id,Accident_Number,Event_Date,Location,Country,Injury_Severity,Aircraft_Damage,Aircraft_Category,Make,Model,...,Total_Fatal_Injuries,Total_Serious_Injuries,Total_Minor_Injuries,Total_Uninjured,Weather_Condition,Broad_phase_of_flight,created_year,created_month,created_day,Total_Injuries
5,20170710X52551,NYC79AA106,1979-09-17,"BOSTON, MA",United States,Non-Fatal,Substantial,Airplane,Douglas,DC9,...,0.0,0.0,1.0,44.0,VMC,Climb,1979,9,17,1.0
7,20020909X01562,SEA82DA022,1982-01-01,"PULLMAN, WA",United States,Non-Fatal,Substantial,Airplane,Cessna,140,...,0.0,0.0,0.0,2.0,VMC,Takeoff,1982,1,1,0.0
8,20020909X01561,NYC82DA015,1982-01-01,"EAST HANOVER, NJ",United States,Non-Fatal,Substantial,Airplane,Cessna,401B,...,0.0,0.0,0.0,2.0,IMC,Landing,1982,1,1,0.0
12,20020917X02148,FTW82FRJ07,1982-01-02,"HOMER, LA",United States,Non-Fatal,Destroyed,Airplane,Bellanca,17-30A,...,0.0,0.0,1.0,0.0,IMC,Cruise,1982,1,2,1.0
13,20020917X02134,FTW82FRA14,1982-01-02,"HEARNE, TX",United States,Fatal,Destroyed,Airplane,Cessna,R172K,...,1.0,0.0,0.0,0.0,IMC,Takeoff,1982,1,2,1.0


## Adding Total Passengers

In [177]:
#Add total passengers column
df['Total_Passengers'] = df['Total_Injuries'] + df['Total_Uninjured']

#Check
df.head()

,Event_Id,Accident_Number,Event_Date,Location,Country,Injury_Severity,Aircraft_Damage,Aircraft_Category,Make,Model,...,Total_Serious_Injuries,Total_Minor_Injuries,Total_Uninjured,Weather_Condition,Broad_phase_of_flight,created_year,created_month,created_day,Total_Injuries,Total_Passengers
5,20170710X52551,NYC79AA106,1979-09-17,"BOSTON, MA",United States,Non-Fatal,Substantial,Airplane,Douglas,DC9,...,0.0,1.0,44.0,VMC,Climb,1979,9,17,1.0,45.0
7,20020909X01562,SEA82DA022,1982-01-01,"PULLMAN, WA",United States,Non-Fatal,Substantial,Airplane,Cessna,140,...,0.0,0.0,2.0,VMC,Takeoff,1982,1,1,0.0,2.0
8,20020909X01561,NYC82DA015,1982-01-01,"EAST HANOVER, NJ",United States,Non-Fatal,Substantial,Airplane,Cessna,401B,...,0.0,0.0,2.0,IMC,Landing,1982,1,1,0.0,2.0
12,20020917X02148,FTW82FRJ07,1982-01-02,"HOMER, LA",United States,Non-Fatal,Destroyed,Airplane,Bellanca,17-30A,...,0.0,1.0,0.0,IMC,Cruise,1982,1,2,1.0,1.0
13,20020917X02134,FTW82FRA14,1982-01-02,"HEARNE, TX",United States,Fatal,Destroyed,Airplane,Cessna,R172K,...,0.0,0.0,0.0,IMC,Takeoff,1982,1,2,1.0,1.0


## Create Make and Model Column - Done

In [178]:
#Create make and model column

df['Make_Model'] = (df['Make'] + ' ' + df['Model'])
df.head()

,Event_Id,Accident_Number,Event_Date,Location,Country,Injury_Severity,Aircraft_Damage,Aircraft_Category,Make,Model,...,Total_Minor_Injuries,Total_Uninjured,Weather_Condition,Broad_phase_of_flight,created_year,created_month,created_day,Total_Injuries,Total_Passengers,Make_Model
5,20170710X52551,NYC79AA106,1979-09-17,"BOSTON, MA",United States,Non-Fatal,Substantial,Airplane,Douglas,DC9,...,1.0,44.0,VMC,Climb,1979,9,17,1.0,45.0,Douglas DC9
7,20020909X01562,SEA82DA022,1982-01-01,"PULLMAN, WA",United States,Non-Fatal,Substantial,Airplane,Cessna,140,...,0.0,2.0,VMC,Takeoff,1982,1,1,0.0,2.0,Cessna 140
8,20020909X01561,NYC82DA015,1982-01-01,"EAST HANOVER, NJ",United States,Non-Fatal,Substantial,Airplane,Cessna,401B,...,0.0,2.0,IMC,Landing,1982,1,1,0.0,2.0,Cessna 401B
12,20020917X02148,FTW82FRJ07,1982-01-02,"HOMER, LA",United States,Non-Fatal,Destroyed,Airplane,Bellanca,17-30A,...,1.0,0.0,IMC,Cruise,1982,1,2,1.0,1.0,Bellanca 17-30A
13,20020917X02134,FTW82FRA14,1982-01-02,"HEARNE, TX",United States,Fatal,Destroyed,Airplane,Cessna,R172K,...,0.0,0.0,IMC,Takeoff,1982,1,2,1.0,1.0,Cessna R172K


**No Nulls**

In [179]:
df['Make_Model'].isna().sum()

0

## Cleaning Engine Type and Number of Engines - Done

In [180]:
#Cleaning up engine type
# Dropping engine types that appear > 20 times

df = df[df.groupby('Engine_Type')['Engine_Type'].transform('count') > 20]

#check
df['Engine_Type'].value_counts()

Reciprocating    15080
Turbo Prop         777
Turbo Fan          658
Unknown            118
Turbo Jet           75
Name: Engine_Type, dtype: int64

In [181]:
#Fill NA's to Unknown
df['Number_of_Engines'].fillna('Unknown', inplace = True)

#Check
df['Number_of_Engines'].isna().sum()

0

## Cleaning Phase of Flight - Done

In [182]:
#Rename column
df.rename(columns = {'Broad_phase_of_flight':'Phase_of_Flight'}, inplace = True)

In [183]:
df['Phase_of_Flight'] 
df['Phase_of_Flight'].isna().sum() 

#Updating null values to Unknown

df['Phase_of_Flight'].fillna('Unknown', inplace = True)

#Check
df['Phase_of_Flight'].value_counts()


Unknown        11655
Landing         1858
Takeoff          968
Cruise           692
Approach         492
Maneuvering      377
Taxi             206
Descent          139
Go-around        135
Climb            127
Standing          51
Other              8
Name: Phase_of_Flight, dtype: int64

## Cleaning Countries

In [184]:
#Convert NA's to Unknown
df['Country'].fillna('Unknown', inplace = True)

#check
df['Country'].isna().sum()

0

## Cleaning Location - Done

In [185]:
#Capitalize all locations 
df['Location'] = df['Location'].apply(lambda x: str(x).upper())

In [186]:
#check Nulls
df['Location'].isna().sum()

(df['Location'] == 'Unknown').sum()

df.head()

,Event_Id,Accident_Number,Event_Date,Location,Country,Injury_Severity,Aircraft_Damage,Aircraft_Category,Make,Model,...,Total_Minor_Injuries,Total_Uninjured,Weather_Condition,Phase_of_Flight,created_year,created_month,created_day,Total_Injuries,Total_Passengers,Make_Model
5,20170710X52551,NYC79AA106,1979-09-17,"BOSTON, MA",United States,Non-Fatal,Substantial,Airplane,Douglas,DC9,...,1.0,44.0,VMC,Climb,1979,9,17,1.0,45.0,Douglas DC9
7,20020909X01562,SEA82DA022,1982-01-01,"PULLMAN, WA",United States,Non-Fatal,Substantial,Airplane,Cessna,140,...,0.0,2.0,VMC,Takeoff,1982,1,1,0.0,2.0,Cessna 140
8,20020909X01561,NYC82DA015,1982-01-01,"EAST HANOVER, NJ",United States,Non-Fatal,Substantial,Airplane,Cessna,401B,...,0.0,2.0,IMC,Landing,1982,1,1,0.0,2.0,Cessna 401B
12,20020917X02148,FTW82FRJ07,1982-01-02,"HOMER, LA",United States,Non-Fatal,Destroyed,Airplane,Bellanca,17-30A,...,1.0,0.0,IMC,Cruise,1982,1,2,1.0,1.0,Bellanca 17-30A
13,20020917X02134,FTW82FRA14,1982-01-02,"HEARNE, TX",United States,Fatal,Destroyed,Airplane,Cessna,R172K,...,0.0,0.0,IMC,Takeoff,1982,1,2,1.0,1.0,Cessna R172K


## Cleaning Purpose of Flight

In [187]:
#change all NA's to Unknown
df['Purpose_of_flight'].fillna('Unknown', inplace = True)

#check
df['Purpose_of_flight'].isna().sum()

0

# Notes

- Review unknown data
- change injury NaN's to 0 (makes sense because total number across rows will show total ppl in flight)

## Check df 

In [188]:
df.head()

,Event_Id,Accident_Number,Event_Date,Location,Country,Injury_Severity,Aircraft_Damage,Aircraft_Category,Make,Model,...,Total_Minor_Injuries,Total_Uninjured,Weather_Condition,Phase_of_Flight,created_year,created_month,created_day,Total_Injuries,Total_Passengers,Make_Model
5,20170710X52551,NYC79AA106,1979-09-17,"BOSTON, MA",United States,Non-Fatal,Substantial,Airplane,Douglas,DC9,...,1.0,44.0,VMC,Climb,1979,9,17,1.0,45.0,Douglas DC9
7,20020909X01562,SEA82DA022,1982-01-01,"PULLMAN, WA",United States,Non-Fatal,Substantial,Airplane,Cessna,140,...,0.0,2.0,VMC,Takeoff,1982,1,1,0.0,2.0,Cessna 140
8,20020909X01561,NYC82DA015,1982-01-01,"EAST HANOVER, NJ",United States,Non-Fatal,Substantial,Airplane,Cessna,401B,...,0.0,2.0,IMC,Landing,1982,1,1,0.0,2.0,Cessna 401B
12,20020917X02148,FTW82FRJ07,1982-01-02,"HOMER, LA",United States,Non-Fatal,Destroyed,Airplane,Bellanca,17-30A,...,1.0,0.0,IMC,Cruise,1982,1,2,1.0,1.0,Bellanca 17-30A
13,20020917X02134,FTW82FRA14,1982-01-02,"HEARNE, TX",United States,Fatal,Destroyed,Airplane,Cessna,R172K,...,0.0,0.0,IMC,Takeoff,1982,1,2,1.0,1.0,Cessna R172K


# Copy to new csv

In [189]:
df.to_csv('Clean_AviationII.csv', index = False)